In [ ]:
from sklearn.discriminant_analysis import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import make_pipeline

# Classify using the average signal in the window 50ms to 300ms
# to focus the classifier on the time interval with best SNR.
clf = make_pipeline(StandardScaler(),
                    LogisticRegression(C=1, solver='liblinear',
                                       multi_class='auto'))
time = all_st_epochs[ival].times
label = all_st_epochs[ival].ch_names
X = all_st_rawdata[ival]['epoch_dat'].mean(axis = 2)
#X = np.einsum('kji->jik', X)
Y = np.array(all_st_rawdata[ival]['metadata'].orient)

#binned_phi = np.digitize(Y, bins = np.array([  0, 22.5, 45, 67.5, 90, 112.5, 135, 157.5 ]) ) # bins collapsing degrees by similarity
binned_phi = np.digitize(Y, bins = np.array([  0, 45, 90, 135 ]) ) # bins collapsing degrees by similarity
G[ival] = binned_phi.copy()
Y = (G[ival]-1)*(180/4)
Y = Y.astype('str')
classes = set(Y)
cv = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)

# Compute confusion matrix for each cross-validation fold
y_pred = np.zeros((len(Y), len(classes)))
for train, test in cv.split(X, Y):
    # Fit
    clf.fit(X[train], Y[train])
    # Probabilistic prediction (necessary for ROC-AUC scoring metric)
    y_pred[test] = clf.predict_proba(X[test])


In [ ]:
from sklearn.metrics import roc_auc_score

confusion = np.zeros((len(classes), len(classes)))
for ii, train_class in enumerate(classes):
    for jj in range(ii, len(classes)):
        confusion[ii, jj] = roc_auc_score(Y == train_class, y_pred[:, jj])
        confusion[jj, ii] = confusion[ii, jj]
#labels = [''] * 5 + ['face'] + [''] * 11 + ['bodypart'] + [''] * 6
fig, ax = plt.subplots(1)
im = ax.matshow(confusion, cmap='RdBu_r', clim=[0.3, 0.7])
ax.set_yticks(range(len(classes)))
#ax.set_yticklabels(labels)
ax.set_xticks(range(len(classes)))
#ax.set_xticklabels(labels, rotation=40, ha='left')
#ax.axhline(11.5, color='k')
#ax.axvline(11.5, color='k')
plt.colorbar(im)
plt.tight_layout()
plt.show()